# importing  require packages

In [1]:
import os
import bs4
import nltk
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from nltk import sent_tokenize
from nltk.corpus import cmudict

# importing data

In [2]:
data=pd.read_excel('cik_list.xlsx')

In [3]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


# Adding link to each cell

In [4]:
data['SECFNAME']='https://www.sec.gov/Archives/'+data['SECFNAME']

In [5]:
data.head(1)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...



# Creating Directories

In [6]:
os.mkdir('dt')
os.mkdir('dt2')
os.mkdir('stopwords')

# List of stopwords files From web

In [7]:
link=['https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_Generic.txt','https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_Names.txt','https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_DatesandNumbers.txt',
       'https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_Geographic.txt','https://www3.nd.edu/~mcdonald/Data/ND_Stop_Words_Currencies.txt']

In [8]:
from time import sleep

# Extracting stopwords data from web

In [9]:
## scraping stop words from web
for i in range(len(link)):
    req=requests.get(link[i],timeout=4)
    sleep(1)
    soup=BeautifulSoup(req.text,'html.parser')
    s=open('stopwords/stopwords_'+str(i)+'.txt',"w+")
    lst=re.findall('\w*[a-zA-Z]',soup.get_text().lower())
    for a in lst:
        s.write(a+' ')
    s.close()

# Extracting Data From web

In [10]:
# scraping data from web
for i in range(len(data)):
    req=requests.get(data['SECFNAME'][i])
    sleep(2.5)
    s=open('dt/'+str(i)+'.txt',"w+",encoding="utf-8")
    s.write(req.text)
    s.close()

# Checking the require sections are present in the data

In [11]:
# checking which sections are present in the data

scores=[]
for i in range(len(data)):
    s=open('dt/'+str(i)+'.txt',"r",encoding="utf-8")
    text=s.read()
    s.close()
    soup=BeautifulSoup(text,'html.parser')
    score=''
    find_text=soup.get_text().lower()
    if "management's discussion and analysis" in find_text:
        score+='1'
    if "quantitative and qualitative disclosures about market risk" in find_text:
        score+='2'
    if "risk factors" in find_text:
        score+='3'
    scores.append(score)

# Saving data into a particular file

In [12]:
for i in range(len(data)):
    x=open('dt/'+str(i)+'.txt','r',encoding='utf-8')
    text=x.read().lower()
    x.close()
    soup=BeautifulSoup(text,'html.parser')

    if '1'in scores[i]:
        text1=''
        for a in soup(text=re.compile("management's discussion and analysis")):
            text1=text1+''+str(a.parent)
            x1=open('dt2/'+str(i)+'_text1.txt',"w+",encoding="utf-8")
            x1.write(text1)
            x1.close()
    if '2' in scores[i]:
        text2=''
        for a in soup(text=re.compile("quantitative and qualitative disclosures about market risk")):
            text2=text2+''+str(a.parent)
            y1=open('dt2/'+str(i)+'_text2.txt',"w+",encoding="utf-8")
            y1.write(text2)
            y1.close()
    if '3'in scores[i]:
        text3=''
        for a in soup(text=re.compile("risk factors")):
            text3=text3+''+str(a.parent)
            z1=open('dt2/'+str(i)+'_text3.txt',"w+",encoding="utf-8")
            z1.write(text3)
            z1.close()

# Importing master directory 

In [14]:
md=pd.read_excel('md.xlsx')

In [15]:
md.head(3)

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,275,1.603442e-08,1.306189e-08,3.665256e-06,82,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.749209e-10,1.028197e-11,1.014208e-08,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,4.664558e-10,1.465871e-10,6.401309e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf


# Creating list of stopwords

In [16]:
# list of stop words

text=''
for i in range(5):
    w=open('stopwords/stopwords_'+str(i)+'.txt','r')
    text+=w.read()
    w.close()
    text+=''
    sw=re.findall('\w*[a-zA-z]',text)

# Creating list of positive and negative words from master directory

In [17]:
# list of positive and negative words

positive_words=list(md[md['Positive']>0]['Word'].apply(lambda f:f.lower()))
negative_words=[]
for i in md[md['Negative']>0]['Word']:
    try:
        negative_words.append(i.lower())
    except:
        pass

# MDA SECTION ANALYSIS

In [18]:
n_words=[]#Length of words
positive_word=[]#positve words 
negative_word=[]#negative words
words=[]# words
total_positive_words=[]#number of positive
total_negative_words=[]#number of negative words
for i in range(len(data)):
    try:
        x=open('dt2/'+str(i)+'_text1.txt','r',encoding="utf-8")
        text=x.read().lower()
        x.close()
        soup=BeautifulSoup(text,'html.parser')
        lst=re.findall('\w*[a-zA-z]',soup.get_text())
        PS=[]
        NS=[]
        
        word=[]
        for j in lst:
            if j not in sw:
                word.append(j)
                if j in positive_words:
                    PS.append(j)
                if j in negative_words:
                     NS.append(j)
        positive_word.append(PS)
        negative_word.append(NS)
        total_positive_words.append(len(PS))
        total_negative_words.append(len(NS))
        words.append(word)
        n_words.append(len(word))
        
    # if the section is absent give none
    except:
        positive_word.append(None)
        negative_word.append(None)
        total_positive_words.append(None)
        total_negative_words.append(None)
        words.append(None)
        n_words.append(None)
        
        

In [19]:
data['mda_positive_score']=total_positive_words
data['mda_negative_score']=total_negative_words

In [20]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,11.0,70.0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,109.0,509.0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN


In [21]:
# polarity scores
polarity=[]
for i in range(len(data)):
    try:
        polarity.append((total_positive_words[i]-total_negative_words[i])/(total_positive_words[i]+total_negative_words[i]+0.000001))
    except:
            polarity.append(None)

In [22]:
data['mda_polarity_score']=polarity

In [23]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0,-0.484330
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,11.0,70.0,-0.728395
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,109.0,509.0,-0.647249
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN


In [24]:
from nltk import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adhya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
sentences=[]#list of lists of sentences in a given document or report for mda section
nsentences=[]#list of number of sentencs in each of the document for mda section
for i in range(len(data)):
    try:
        x=open('dt2/'+str(i)+'_text1.txt',"r",encoding='utf-8')
        text=x.read()
        x.close()
        soup=BeautifulSoup(text,'html.parser')
        lst_sentences=sent_tokenize(soup.get_text())
        sentences.append(lst_sentences)
        nsentences.append(len(lst_sentences))
    except:
        sentences.append(None)
        nsentences.append(None)

In [26]:
avg_sen_len=[]#average sentence lengths for mda section of each document
for i in range(len(data)):
    try:
        avg_sen_len.append(n_words[i]/nsentences[i])
    except:
        avg_sen_len.append(None)

In [27]:
data['mda_average_sentence_length']=avg_sen_len
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0,-0.484330,18.984783
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,11.0,70.0,-0.728395,21.922078
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,109.0,509.0,-0.647249,20.245283
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN


In [28]:
# creating functions for complexwords in the sentences
cd=cmudict.dict()

In [29]:
def n_syl(word):
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in cd[word.lower()]][0]
    except KeyError:
        #if word not found in cmudict
        return syllables(word)

def syllables(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count

In [30]:
comp_words=[]#list of percentages of complex words i.e. words having more than two syllables for mda section
n_comp_words=[]#list of number of complex words for each document for mda section
for i in range(len(data)):
    n_cmplx=0
    try:
        for j in words[i]:
            if n_syl(j)>2:
                n_cmplx+=1
        comp_words.append(n_cmplx*100/n_words[i])
        n_comp_words.append(n_cmplx)
    except:
        comp_words.append(None)
        n_comp_words.append(None)

In [31]:
data['mda_percentage_of_complex_words']=comp_words

In [32]:
data.head(1)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0,-0.48433,18.984783,48.700332


In [33]:
fog_index=[] #list of fog indexes for mda section
for i in range(len(data)):
    try:
        fog_index.append(0.4*(comp_words[i]+avg_sen_len[i]))
    except:
        fog_index.append(None)

In [34]:
data['mda_complex_word_count']=n_comp_words

In [35]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_complex_word_count
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0,-0.484330,18.984783,48.700332,8506.0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,11.0,70.0,-0.728395,21.922078,45.319905,765.0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,NaN,NaN
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,109.0,509.0,-0.647249,20.245283,48.860459,5767.0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# uncertain and constraining words
cons=pd.read_excel('constraining_dictionary.xlsx')
uncertain=pd.read_excel('uncertainty_dictionary.xlsx')

In [37]:
cons_words=[x.lower() for x in cons['Word']]
uncertain_words=[x.lower() for x in uncertain['Word']]

In [38]:
constraining_words=[]
n_constraining=[] 
uncertain_words=[] 
n_uncertainty=[] 
for i in range(len(data)):
    lst_cons=[]
    lst_unce=[]
    try:
        for j in words[i]:
            if j in cons_words:
                lst_cons.append(j)
            if j in uncertain_words:
                lst_unce.append(j)
        constraining_words.append(lst_cons)
        n_constraining.append(len(lst_cons))
        uncertain_words.append(lst_unce)
        n_uncertainty.append(len(lst_unce))
    except:
        constraining_words.append(None)
        n_constraining.append(None)
        uncertain_words.append(None)
        n_uncertainty.append(None)

In [39]:
data['mda_uncertainty_score']=n_uncertainty
data['mda_constraining_score']=n_constraining

In [40]:
positive_word_proportions=[]
negative_word_proportions=[]
constraining_word_proportions=[]
uncertainty_word_proportions=[]
for i in range(len(data)):
    try:
        positive_word_proportions.append(total_positive_words[i]/n_words[i])
        negative_word_proportions.append(total_negative_words[i]/n_words[i])
        constraining_word_proportions.append(n_constraining[i]/n_words[i])
        uncertainty_word_proportions.append(n_uncertainty[i]/n_words[i])
    except:
        positive_word_proportions.append(None)
        negative_word_proportions.append(None)
        constraining_word_proportions.append(None)
        uncertainty_word_proportions.append(None)

In [41]:
data['mda_positive_word_proportion']=positive_word_proportions
data['mda_negative_word_proportion']=negative_word_proportions
data['mda_constraining_word_proportion']=constraining_word_proportions
data['mda_uncertainty_word_proportion']=uncertainty_word_proportions

In [42]:
data.head(2)

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_complex_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_constraining_word_proportion,mda_uncertainty_word_proportion
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0,-0.484330,18.984783,48.700332,8506.0,0.0,188.0,0.010363,0.029829,0.010764,0.0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,11.0,70.0,-0.728395,21.922078,45.319905,765.0,0.0,4.0,0.006517,0.041469,0.002370,0.0


# quantitative and qualitative disclosures about market risk Analysis


In [43]:
n_words=[]#Length of words
positive_word=[]#positve words 
negative_word=[]#negative words
words=[]# words
total_positive_words=[]#number of positive
total_negative_words=[]#number of negative words
for i in range(len(data)):
    try:
        x=open('dt2/'+str(i)+'_text2.txt','r',encoding="utf-8")
        text=x.read().lower()
        x.close()
        soup=BeautifulSoup(text,'html.parser')
        lst=re.findall('\w*[a-zA-z]',soup.get_text())
        PS=[]
        NS=[]
        word=[]
        for j in lst:
            if j not in sw:
                word.append(j)
                if j in positive_words:
                    PS.append(j)
                if j in negative_words:
                     NS.append(j)
        positive_word.append(PS)
        negative_word.append(NS)
        total_positive_words.append(len(PS))
        total_negative_words.append(len(NS))
        words.append(word)
        n_words.append(len(word))
        
    # if the section is absent give none
    except:
        positive_word.append(None)
        negative_word.append(None)
        total_positive_words.append(None)
        total_negative_words.append(None)
        words.append(None)
        n_words.append(None)

In [44]:
data['qqdmr_positive_score']=total_positive_words
data['qqdmr_negative_score']=total_negative_words

In [45]:
# polarity scores
polarity=[]
for i in range(len(data)):
    try:
        polarity.append((total_positive_words[i]-total_negative_words[i])/(total_positive_words[i]+total_negative_words[i]+0.000001))
    except:
            polarity.append(None)

In [46]:
data['qqdmr_polarity_score']=polarity

In [47]:
data

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,mda_complex_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_constraining_word_proportion,mda_uncertainty_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0,-0.484330,18.984783,...,8506.0,0.0,188.0,0.010363,0.029829,0.010764,0.0,NaN,NaN,NaN
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,11.0,70.0,-0.728395,21.922078,...,765.0,0.0,4.0,0.006517,0.041469,0.002370,0.0,NaN,NaN,NaN
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,109.0,509.0,-0.647249,20.245283,...,5767.0,0.0,175.0,0.009235,0.043125,0.014827,0.0,NaN,NaN,NaN
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3662,SUNBEAM CORP/FL/,199811,1998-11-25,10-Q/A,https://www.sec.gov/Archives/edgar/data/3662/0...,42.0,304.0,-0.757225,17.814126,...,2309.0,0.0,38.0,0.008765,0.063439,0.007930,0.0,NaN,NaN,NaN
6,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,66.0,193.0,-0.490347,18.240157,...,2263.0,0.0,49.0,0.014246,0.041658,0.010576,0.0,NaN,NaN,NaN
7,3662,SUNBEAM CORP/FL/,199812,1998-12-22,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,58.0,151.0,-0.444976,18.081818,...,1952.0,0.0,44.0,0.014580,0.037959,0.011061,0.0,NaN,NaN,NaN
8,3662,SUNBEAM CORP/FL/,199903,1999-03-31,NT 10-K,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3662,SUNBEAM CORP/FL/,199905,1999-05-11,10-K,https://www.sec.gov/Archives/edgar/data/3662/0...,977.0,4853.0,-0.664837,18.295364,...,52571.0,0.0,1316.0,0.008906,0.044239,0.011996,0.0,NaN,NaN,NaN


In [48]:
sentences=[]#list of lists of sentences in a given document or report for mda section
nsentences=[]#list of number of sentencs in each of the document for mda section
for i in range(len(data)):
    try:
        x=open('dt2/'+str(i)+'_text2.txt',"r",encoding='utf-8')
        text=x.read()
        x.close()
        soup=BeautifulSoup(text,'html.parser')
        lst_sentences=sent_tokenize(soup.get_text())
        sentences.append(lst_sentences)
        nsentences.append(len(lst_sentences))
    except:
        sentences.append(None)
        nsentences.append(None)

In [49]:
avg_sen_len=[]#average sentence lengths for mda section of each document
for i in range(len(data)):
    try:
        avg_sen_len.append(n_words[i]/nsentences[i])
    except:
        avg_sen_len.append(None)

In [50]:
data['qqdmr_average_sentence_length']=avg_sen_len

In [51]:
comp_words=[]#list of percentages of complex words i.e. words having more than two syllables for mda section
n_comp_words=[]#list of number of complex words for each document for mda section
for i in range(len(data)):
    n_cmplx=0
    try:
        for j in words[i]:
            if n_syl(j)>2:
                n_cmplx+=1
        comp_words.append(n_cmplx*100/n_words[i])
        n_comp_words.append(n_cmplx)
    except:
        comp_words.append(None)
        n_comp_words.append(None)

In [52]:
data['qqdmr_percentage_of_complex_words']=n_comp_words

In [53]:
fog_index=[] #list of fog indexes for mda section
for i in range(len(data)):
    try:
        fog_index.append(0.4*(comp_words[i]+avg_sen_len[i]))
    except:
        fog_index.append(None)

In [54]:
data['qqdmr_fog_index']=fog_index

In [55]:
constraining_words=[]
n_constraining=[] 
uncertain_words=[] 
n_uncertainty=[] 
for i in range(len(data)):
    lst_cons=[]
    lst_unce=[]
    try:
        for j in words[i]:
            if j in cons_words:
                lst_cons.append(j)
            if j in uncertain_words:
                lst_unce.append(j)
        constraining_words.append(lst_cons)
        n_constraining.append(len(lst_cons))
        uncertain_words.append(lst_unce)
        n_uncertainty.append(len(lst_unce))
    except:
        constraining_words.append(None)
        n_constraining.append(None)
        uncertain_words.append(None)
        n_uncertainty.append(None)

In [56]:
data['qqdmr_uncertainty_score']=n_uncertainty
data['qqdmr_constraining_score']=n_constraining

In [57]:
positive_word_proportions=[]
negative_word_proportions=[]
constraining_word_proportions=[]
uncertainty_word_proportions=[]
for i in range(len(data)):
    try:
        positive_word_proportions.append(total_positive_words[i]/n_words[i])
        negative_word_proportions.append(total_negative_words[i]/n_words[i])
        constraining_word_proportions.append(n_constraining[i]/n_words[i])
        uncertainty_word_proportions.append(n_uncertainty[i]/n_words[i])
    except:
        positive_word_proportions.append(None)
        negative_word_proportions.append(None)
        constraining_word_proportions.append(None)
        uncertainty_word_proportions.append(None)

In [58]:
data['qqdmr_positive_word_proportion']=positive_word_proportions
data['qqdmr_negative_word_proportion']=negative_word_proportions
data['qqdmr_constraining_word_proportion']=constraining_word_proportions
data['qqdmr_uncertainty_word_proportion']=uncertainty_word_proportions

# Risk Factors Analysis

In [59]:
n_words=[]#Length of words
positive_word=[]#positve words 
negative_word=[]#negative words
words=[]# words
total_positive_words=[]#number of positive
total_negative_words=[]#number of negative words
for i in range(len(data)):
    try:
        x=open('dt2/'+str(i)+'_text3.txt','r',encoding="utf-8")
        text=x.read().lower()
        x.close()
        soup=BeautifulSoup(text,'html.parser')
        lst=re.findall('\w*[a-zA-z]',soup.get_text())
        PS=[]
        NS=[]
        word=[]
        for j in lst:
            if j not in sw:
                word.append(j)
                if j in positive_words:
                    PS.append(j)
                if j in negative_words:
                     NS.append(j)
        positive_word.append(PS)
        negative_word.append(NS)
        total_positive_words.append(len(PS))
        total_negative_words.append(len(NS))
        words.append(word)
        n_words.append(len(word))
        
    # if the section is absent give none
    except:
        positive_word.append(None)
        negative_word.append(None)
        total_positive_words.append(None)
        total_negative_words.append(None)
        words.append(None)
        n_words.append(None)

In [60]:
data['rf_positive_score']=total_positive_words
data['rf_negative_score']=total_negative_words

In [61]:
# polarity scores
polarity=[]
for i in range(len(data)):
    try:
        polarity.append((total_positive_words[i]-total_negative_words[i])/(total_positive_words[i]+total_negative_words[i]+0.000001))
    except:
            polarity.append(None)

In [62]:
data['rf_polarity_score']=polarity

In [63]:
sentences=[]#list of lists of sentences in a given document or report for mda section
nsentences=[]#list of number of sentencs in each of the document for mda section
for i in range(len(data)):
    try:
        x=open('dt2/'+str(i)+'_text3.txt',"r",encoding='utf-8')
        text=x.read()
        x.close()
        soup=BeautifulSoup(text,'html.parser')
        lst_sentences=sent_tokenize(soup.get_text())
        sentences.append(lst_sentences)
        nsentences.append(len(lst_sentences))
    except:
        sentences.append(None)
        nsentences.append(None)

In [64]:
avg_sen_len=[]#average sentence lengths for mda section of each document
for i in range(len(data)):
    try:
        avg_sen_len.append(n_words[i]/nsentences[i])
    except:
        avg_sen_len.append(None)

In [65]:
data['rf_average_sentence_length']=avg_sen_len

In [66]:
comp_words=[]#list of percentages of complex words i.e. words having more than two syllables for mda section
n_comp_words=[]#list of number of complex words for each document for mda section
for i in range(len(data)):
    n_cmplx=0
    try:
        for j in words[i]:
            if n_syl(j)>2:
                n_cmplx+=1
        comp_words.append(n_cmplx*100/n_words[i])
        n_comp_words.append(n_cmplx)
    except:
        comp_words.append(None)
        n_comp_words.append(None)

In [67]:
data['rf_percentage_of_complex_words']=n_comp_words

In [68]:
fog_index=[] #list of fog indexes for mda section
for i in range(len(data)):
    try:
        fog_index.append(0.4*(comp_words[i]+avg_sen_len[i]))
    except:
        fog_index.append(None)

In [69]:
data['rf_fog_index']=fog_index

In [70]:
constraining_words=[]
n_constraining=[] 
uncertain_words=[] 
n_uncertainty=[] 
for i in range(len(data)):
    lst_cons=[]
    lst_unce=[]
    try:
        for j in words[i]:
            if j in cons_words:
                lst_cons.append(j)
            if j in uncertain_words:
                lst_unce.append(j)
        constraining_words.append(lst_cons)
        n_constraining.append(len(lst_cons))
        uncertain_words.append(lst_unce)
        n_uncertainty.append(len(lst_unce))
    except:
        constraining_words.append(None)
        n_constraining.append(None)
        uncertain_words.append(None)
        n_uncertainty.append(None)

In [71]:
data['rf_uncertainty_score']=n_uncertainty
data['rf_constraining_score']=n_constraining

In [72]:
positive_word_proportions=[]
negative_word_proportions=[]
constraining_word_proportions=[]
uncertainty_word_proportions=[]
for i in range(len(data)):
    try:
        positive_word_proportions.append(total_positive_words[i]/n_words[i])
        negative_word_proportions.append(total_negative_words[i]/n_words[i])
        constraining_word_proportions.append(n_constraining[i]/n_words[i])
        uncertainty_word_proportions.append(n_uncertainty[i]/n_words[i])
    except:
        positive_word_proportions.append(None)
        negative_word_proportions.append(None)
        constraining_word_proportions.append(None)
        uncertainty_word_proportions.append(None)

In [73]:
data['rf_positive_word_proportion']=positive_word_proportions
data['rf_negative_word_proportion']=negative_word_proportions
data['rf_constraining_word_proportion']=constraining_word_proportions
data['rf_uncertainty_word_proportion']=uncertainty_word_proportions

In [75]:
total_constraining=[]
for i in range(len(data)):
    s=open('dt/'+str(i)+'.txt',"r",encoding='utf-8')
    text=s.read().lower()
    s.close()
    soup=BeautifulSoup(text,'html.parser')
    words=re.findall('\w*[a-zA-Z]',soup.get_text())
    n_const=0
    for j in words:
        if j in constraining_words:
            n_const+=1
    total_constraining.append(n_constraining)

In [76]:
data['constraining_words_whole_report']=constraining_scores

In [77]:
data.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_constraining_word_proportion,rf_uncertainty_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,181.0,521.0,-0.484330,18.984783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,11.0,70.0,-0.728395,21.922078,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,109.0,509.0,-0.647249,20.245283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [78]:
data.to_excel('output.xlsx',index=False)